Рассматривается следующая задача:

$\\ 25λ\frac{\partial^{2}\phi}{\partial x^{2}} + λ\frac{\partial^{2}\phi}{\partial y^{2}} = \frac{\partial\phi}{\partial \tau} \qquad x,y,t \in [0,1] $

$ \phi(x,y,0) = cos(\pi x)sin(5\pi y) \\ \phi(x,0,t) = \phi(x,1,t) = 0, \qquad x,t \in [0,1] \\ \phi(0,y,t) = sin(5\pi y)exp(-5\pi^{2}t),\quad \phi(1,y,t) = -sin(5\pi y)exp(-5\pi^{2}t),\qquad y,t \in [0,1]$

 решаем ее численными методами.

In [1]:
import math
import numpy as np

In [2]:
λ = 1e-4
π = math.pi

Начальные условия:

In [3]:
def φt0(x,y):
    return np.sin(5*π*y) * np.cos(π*x)

Граничное условие:

In [4]:
def φx0(x,t):
    return 0.0
def φx1(x,t):
    return 0.0
def φy0(y,t):
    return (np.sin(5*π*y) * np.exp(-5*π*π*λ*t))
def φy1(y,t):
    return (-np.sin(5*π*y) * np.exp(-5*π*π*λ*t))

Аналитическое решение уравнения:

In [5]:
def analytical(x,y,t):
    return np.sin(5*π*y) * np.cos(π*x) * np.exp(-5*π*π*λ*t)

In [8]:
NX = int(input("Enter NX:"))
NY = NX
## Генерация сетки
hx = 1/(NX - 1)
hy = 1/(NY - 1)
X = np.arange(0.0,1.0,hx)
Y = np.arange(0.0,1.0,hy)
num = NX * NY
## Задание нач.данных
τ = 1e-3
T = 1
ε = 1e-2
## Нумерация по строкам
φ = np.zeros(num)
for i in range(num):
    φ[i] = φt0(hx * (i % NX), hy * (i / NY))

Enter NX:11
